# Peatland Protection Interactive Map
_The script below uses various packages to load, modify and generate an interactive map which allows the user to engage and gain understanding of the peatlands residing in Northern Ireland. 
This map increases the awareness of various residents to avoid hindrance on unprotected areas._
## Get Started!

In [ ]:
"""
Peatland Protection  Interactive Map
Shows protected/unprotected peatlands within Northern Ireland with clickable popups.
Farms and trails are also visible within the map.
"""
import geopandas as gpd                    # Used for spatial data handling
import pandas as pd                        # Used for dataframes and joining
import folium                              # Used for interactive mapping
from shapely.geometry import LineString    # Used for distance line drawing


In [ ]:
def load_and_prepare_data():
    """
    Load and reproject all spatial datasets to EPSG:4326.
    """
    peat = gpd.read_file("datafiles/Peatland/Priority_Habitats_-_Peatland.shp").to_crs(epsg=4326)    # Peatland polygons
    assi = gpd.read_file("datafiles/ASSI/ASSI.shp").to_crs(epsg=4326)                                # Protected ASSI areas
    aonb = gpd.read_file("datafiles/AONB/AONB.shp").to_crs(epsg=4326)                                # AONB areas
    trails = gpd.read_file("datafiles/Trails/offroadtrailsni.shp").to_crs(epsg=4326)                 # Trails networks
    scheme_data = gpd.read_file("datafiles/Farm/SchemeData.shp").to_crs(epsg=4326)                   # Farm scheme data

    return peat, assi, aonb, trails, scheme_data
peat, assi, aonb, trails, scheme_data = load_and_prepare_data()

In [ ]:
"""
  Confirms peat is in EPSG:4326 for web display
"""
peat.crs

In [ ]:
"""
Confirms assi is in EPSG:4326 for web display
"""
assi.crs

In [ ]:
"""
Confirms aonb is in EPSG:4326 for web display
"""
aonb.crs

In [ ]:
"""
Confirms trails is in EPSG:4326 for web display
"""
trails.crs

In [ ]:
"""
Confirms scheme_data is in EPSG:4326 for web display
"""
scheme_data.crs

In [ ]:
"""
Previews the first 10 rows of the peat GeoDataFrame to review structure and attributes
"""
peat.head(10)

In [ ]:
"""
Access and display the row of peat GeoDataFrame with index label 10
"""
peat.loc[10]

In [ ]:
peat['Area_Hecta'].sum() # Gives area of all peat in hectares

In [ ]:
peat['Area_Sq_Km'].sum() # Gives area of all peat in square kilometers 

In [ ]:
peat['Shape__Are'].sum() # Gives sum of all peat within "Shape__Are"

In [ ]:
"""
Previews the first 10 rows of the aonb GeoDataFrame to review structure and attributes
"""
aonb.head(10)

In [ ]:
"""
Preview the first 10 rows of the assi GeoDataFrame to review structure and attributes
"""
assi.head(10)

In [ ]:
"""
Preview the first 10 rows of the trails GeoDataFrame to review structure and attributes
"""
trails.head(10)

In [ ]:
"""
Preview the first 10 rows of the scheme_data GeoDataFrame to review structure and attributes
"""
scheme_data.head(10)

In [ ]:
def classify_peatland(peat, assi, aonb):
    """
    Intersects peat with ASSI and AONB to create the protected layer.
    The peat remaining becomes the unprotected layer.
    """

    # Adapted for intersection from: https://geopandas.org/en/stable/docs/user_guide/set_operations.html
    
    peat_assi = gpd.overlay(peat, assi, how='intersection')    # peat joined inside ASSI
    peat_aonb = gpd.overlay(peat, aonb, how='intersection')    # peat joined inside AONB

    protected_peat = gpd.GeoDataFrame(pd.concat([peat_assi, peat_aonb], ignore_index=True), crs=peat.crs).copy()  # merges peat, ASSI and AONB
    protected_peat["Area_ha"] = protected_peat.to_crs(epsg=3035).geometry.area / 10000  # calculate area in hectares in EPSG:3035

    unioned = protected_peat.geometry.union_all()  # create single protected area union
    unprotected_peat = peat[~peat.geometry.intersects(unioned)].copy()  # combines everything else outside protected peat
    unprotected_peat["Area_ha"] = unprotected_peat.to_crs(epsg=3035).geometry.area / 10000  # calculate area in hectares in EPSG:3035

    return protected_peat, unprotected_peat


In [ ]:
def create_interactive_map(protected_peat, unprotected_peat, trails, scheme_data):
    """
    Creates an interactive Folium map with multiple toggleable layers for:
    - peat (Protected peatlands)
    - peat (Unprotected peatlands)
    - trails (Off-road trails)
    - scheme Data (Farm (scheme data))
    Includes popups and tooltips for each layer
    """
    center = protected_peat.geometry.union_all().centroid  # gets the center of map based on peatlands

    m = folium.Map(location=[center.y, center.x], zoom_start=8)  # base maps setup
    

    # Adapted for lamda from:https://github.com/python-visualization/folium/issues/318
    # Adapted for layercontrol from: https://python-visualization.github.io/folium/latest/user_guide/ui_elements/layer_control.html
    
    #        - Protected Peat-
    
    fg_protected = folium.FeatureGroup(name="Protected Peatlands", show=True)  # creates the layer group for toggle

    for _, row in protected_peat.iterrows():  # proceses attributes and geometry within each protected peatland polygons
        name = row.get("ASSI_NAME") or row.get("AONB_NAME") or row.get("NAME") or "Protected Peatland"  # Assigns the name
        area = round(row["Area_ha"], 2)  # retrives area 
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5},  # visually styles each polygon
            tooltip=name,  # Displays name of the peatland as a label when hoverring over it
            popup=folium.Popup(f"<b>{name}</b><br>Area: {area} ha", max_width=300)  # click shows popup with feature's of name and size.
        ).add_to(fg_protected)  #adds to layer to allow it to be turned on/off

    fg_protected.add_to(m)  # places into map
    

    #        - Unprotected Peat -
    
    fg_unprotected = folium.FeatureGroup(name="Unprotected Peatlands", show=True)  # creates the layer group for toggle

    for _, row in unprotected_peat.iterrows(): # proceses attributes and geometry within each unprotected peatland polygons
        name = row.get("ASSI_NAME") or row.get("AONB_NAME") or row.get("NAME") or "Protected Peatland" # Assigns the name
        area = round(row["Area_ha"], 2) # retrives area 
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'darkviolet', 'color': 'darkviolet', 'weight': 1, 'fillOpacity': 0.4},  # visually styles each polygon
            tooltip="Unprotected Peatland", # Displays name of the peatland as a label when hoverring over it
            popup=folium.Popup(f"<b>Unprotected Peatland</b><br>Area: {area} ha", max_width=300) # click shows popup with feature's of name and size.
        ).add_to(fg_unprotected) #adds to layer to allow it to be turned on/off

    fg_unprotected.add_to(m) # places into map
    

    #         - Trails -
    
    fg_trails = folium.FeatureGroup(name="Off-Road Trails", show=True)  # creates the layer group for toggle

    folium.GeoJson(
        trails,
        style_function=lambda x: {'color': 'blue', 'weight': 1.2},  # visually styles each polygon
        tooltip=folium.GeoJsonTooltip(
        fields=['Name', 'Length'],
            aliases=['Trail Name:', 'Length (km):'],
            localize=True
        ),
        popup=folium.GeoJsonPopup(      #displays the name and length of each trail
            fields=['Name', 'Length'],
            aliases=['<b>Trail Name:</b>', '<b>Length (km):</b>'],
            localize=True,
            max_width=300
        )
    ).add_to(fg_trails) #adds to layer to allow it to be turned on/off
    fg_trails.add_to(m) # places into map

    
    #        - Farms (Scheme Data) -
    
    
    fg_farms = folium.FeatureGroup(name="Farms (Scheme Data)", show=True)  # creates the layer group for toggle

    for _, row in scheme_data.iterrows():  # proceses attributes and geometry within each farm polygons
        if row.geometry.is_empty:
            continue
        label = row.get("FARM_ID") or row.get("FARMREF") or row.get("Farm_Name") or "Farm"
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'weight': 1, 'fillOpacity': 0.2},  # visually styles each polygon
            tooltip=label, # Displays label of the farm as a label when hoverring over it
            popup=folium.Popup(f"<b>Farm ID:</b> {label}", max_width=300)  # click shows popup with feature's of name and size.
        ).add_to(fg_farms) #adds to layer to allow it to be turned on/off

    fg_farms.add_to(m) # places into map
    

    #         - Add Layer Control -
    
    folium.LayerControl().add_to(m) # places into map
    
    return m


In [ ]:
"""
Load Peatland Protection Map and save it as an html file
"""

# Load all datasets
peat, assi, aonb, trails, scheme_data = load_and_prepare_data()

# classifies protected status
protected_peat, unprotected_peat = classify_peatland(peat, assi, aonb)

# Builds the map with layers
m = create_interactive_map(protected_peat, unprotected_peat, trails, scheme_data)

# Save map as HTML file
m.save("peatland_protection_map_interactive.html")
print("Map saved as: peatland_protection_map_interactive.html")



##### _You should now see an interactive map. Explore the map and increase your knowledge._

#### _Thank you and Enjoy._